# Exploring Conversation Chains and Memory with LCEL

## Install OpenAI, and LangChain dependencies

In [ ]:
!pip install -qq langchain
!pip install -qq langchain-openai
!pip install -qq langchain-community
!pip install -qq langchain-chroma

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.50.0 requires tokenizers<0.22,>=0.21, but you have tokenizers 0.20.3 which is incompatible.
langchain-huggingface 0.3.0 requires huggingface-hub>=0.30.2, but you have huggingface-hub 0.27.1 which is incompatible.
langchain-huggingface 0.3.0 requires langchain-core<1.0.0,>=0.3.65, but you have langchain-core 0.3.63 which is incompatible.


## Enter Open AI API Key

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

True

## Load Connection to LLM

Here we create a connection to ChatGPT to use later in our chains

In [3]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name='gpt-4o-mini', temperature=0)

## Working with LangChain Chains

Using an LLM in isolation is fine for simple applications, but more complex applications require chaining LLMs - either with each other or with other components. Also running on multiple data points can be done easily with chains.

Chain's are the legacy interface for "chained" applications. We define a Chain very generically as a sequence of calls to components, which can include other chains.

Here we will be using LCEL chains exclusively

### The Problem with Simple LLM Chains

Simple LLM Chains cannot keep a track of past conversation history

In [4]:
from langchain_core.prompts import ChatPromptTemplate,PromptTemplate

prompt_text = """{query}"""
prompt = ChatPromptTemplate.from_template(prompt_text)

llm_chain = prompt | chatgpt

response = llm_chain.invoke({"query": "What is the capital of France?"})
print(response.content)

The capital of France is Paris.


In [6]:
response = llm_chain.invoke({"query":"And what about India?"})
print(response.content)

Could you please provide more context or specify what aspect of India you are interested in? India is a vast country with a rich history, diverse culture, and significant developments in various fields such as politics, economy, technology, and social issues. Let me know what specific information you are looking for!


## Working with ConversationBufferMemory

In [7]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

SYS_PROMPT = """Act as a helpful assistant and give brief answers"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", SYS_PROMPT),
        MessagesPlaceholder(variable_name="history"), # This is where the conversation history will be stored
        ("human", "{query}"),
    ]
)

memory = ConversationBufferMemory(return_messages=True)

/var/folders/8v/xkrl1q210t5_4t4hvbx286800000gp/T/ipykernel_5092/2476208786.py:14: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True)


In [8]:
print(type(memory))

<class 'langchain.memory.buffer.ConversationBufferMemory'>


In [9]:
# function to get historical conversation messages from the memory
memory.load_memory_variables({})

{'history': []}

In [11]:
print(memory.load_memory_variables({})['history'])

[]


In [12]:
# lets create a function now which returns the list of messages from memory
def get_memory_messages(query):
    return memory.load_memory_variables(query)['history']

print(get_memory_messages('What are the first four colors of a rainbow?'))

[]


In [13]:
type(RunnableLambda(get_memory_messages))

langchain_core.runnables.base.RunnableLambda

In [14]:
# testing out the function with a runnable lambda which will go into our chain
# this returns the history but we also need to send our current query to the prompt
print(RunnableLambda(get_memory_messages).invoke({'query': 'What are the first four colors of a rainbow?'}))

[]


In [16]:
# We use RunnablePassthrough.assign() to combine the current input with memory history
# This allows us to pass the query unchanged while adding the conversation history
# The .assign() method adds a new key 'history' to the input dictionary
# RunnableLambda(get_memory_messages) extracts the conversation history from memory
RunnablePassthrough.assign(
        history=RunnableLambda(get_memory_messages)
    ).invoke({'query': 'What are the first four colors of a rainbow1?','query2':"sourav"})

{'query': 'What are the first four colors of a rainbow1?',
 'query2': 'sourav',
 'history': []}

In [17]:
chain1 = {
    'query': RunnableLambda(lambda x: x['query2'])
} |RunnablePassthrough.assign(
        history=RunnableLambda(get_memory_messages)
    )
chain1.invoke({'query1': 'What are the first four colors of a rainbow1?','query2':"sourav"})

{'query': 'sourav', 'history': []}

In [18]:
chain2 = {
    'query': RunnableLambda(lambda x: x['query1'])
} |RunnablePassthrough.assign(
        history=RunnableLambda(get_memory_messages)
    )

chain2.invoke({'query1': 'What are the first four colors of a rainbow1?','query2':"sourav"})

{'query': 'What are the first four colors of a rainbow1?', 'history': []}

In [19]:
runnable = RunnableLambda(lambda x: x['query1'])
runnable.invoke({'query1': 'What are the first four colors of a rainbow1?', 'query2': "sourav"})

'What are the first four colors of a rainbow1?'

In [21]:
def fake_llm(prompt: str) -> str: # Fake LLM for the example
    return "completion"

chain = {
    'query': RunnableLambda(lambda x: x['query']), 
    'llm1':  fake_llm,
    'llm2':  fake_llm,
} | RunnablePassthrough.assign(
    history=RunnableLambda(get_memory_messages)
)

chain.invoke({'query': 'What are the first four colors of a rainbow1?'})

{'query': 'What are the first four colors of a rainbow1?',
 'llm1': 'completion',
 'llm2': 'completion',
 'history': []}

In [22]:
# creating our conversation chain now
def get_memory_messages(query):
    return memory.load_memory_variables(query)['history']

conversation_chain = (
    RunnablePassthrough.assign(
        history=RunnableLambda(get_memory_messages)
    ) # sends current query (input by user at runtime) and history messages to next step
      |
    prompt # creates prompt using the previous two variables
      |
    chatgpt # generates response using the prompt from previous step
)

In [23]:
query = {'query': 'What are the first four colors of a rainbow?'}
response = conversation_chain.invoke(query)
print(response)

content='The first four colors of a rainbow are red, orange, yellow, and green.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 30, 'total_tokens': 47, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'stop', 'logprobs': None} id='run--b2510428-ef84-4254-a0ea-e768dae4d4ac-0' usage_metadata={'input_tokens': 30, 'output_tokens': 17, 'total_tokens': 47, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [26]:
response.dict() #deprecated
response.model_dump()

/var/folders/8v/xkrl1q210t5_4t4hvbx286800000gp/T/ipykernel_5092/2263478525.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  response.dict() #deprecated


{'content': 'The first four colors of a rainbow are red, orange, yellow, and green.',
 'additional_kwargs': {'refusal': None},
 'response_metadata': {'token_usage': {'completion_tokens': 17,
   'prompt_tokens': 30,
   'total_tokens': 47,
   'completion_tokens_details': {'accepted_prediction_tokens': 0,
    'audio_tokens': 0,
    'reasoning_tokens': 0,
    'rejected_prediction_tokens': 0},
   'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
  'model_name': 'gpt-4o-mini-2024-07-18',
  'system_fingerprint': 'fp_34a54ae93c',
  'finish_reason': 'stop',
  'logprobs': None},
 'type': 'ai',
 'name': None,
 'id': 'run--b2510428-ef84-4254-a0ea-e768dae4d4ac-0',
 'example': False,
 'tool_calls': [],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens': 30,
  'output_tokens': 17,
  'total_tokens': 47,
  'input_token_details': {'audio': 0, 'cache_read': 0},
  'output_token_details': {'audio': 0, 'reasoning': 0}}}

In [27]:
print(response.content)

The first four colors of a rainbow are red, orange, yellow, and green.


In [28]:
memory.load_memory_variables({})

{'history': []}

In [29]:
memory.save_context(query, {"output": response.content})

In [30]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='What are the first four colors of a rainbow?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The first four colors of a rainbow are red, orange, yellow, and green.', additional_kwargs={}, response_metadata={})]}

In [31]:
query = {'query': 'and the other 3?'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

The other three colors of a rainbow are blue, indigo, and violet.


In [32]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='What are the first four colors of a rainbow?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The first four colors of a rainbow are red, orange, yellow, and green.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='and the other 3?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The other three colors of a rainbow are blue, indigo, and violet.', additional_kwargs={}, response_metadata={})]}

In [33]:
query = {'query': 'Explain AI in 2 bullet points'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

- **Definition**: Artificial Intelligence (AI) refers to the simulation of human intelligence in machines programmed to think and learn, enabling them to perform tasks that typically require human cognitive functions, such as problem-solving, understanding language, and recognizing patterns.

- **Applications**: AI is used in various fields, including healthcare (diagnosis and treatment recommendations), finance (fraud detection and algorithmic trading), and everyday technology (virtual assistants and recommendation systems).


In [37]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='What are the first four colors of a rainbow?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The first four colors of a rainbow are red, orange, yellow, and green.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='and the other 3?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The other three colors of a rainbow are blue, indigo, and violet.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Explain AI in 2 bullet points', additional_kwargs={}, response_metadata={}),
  AIMessage(content='- **Definition**: Artificial Intelligence (AI) refers to the simulation of human intelligence in machines programmed to think and learn, enabling them to perform tasks that typically require human cognitive functions, such as problem-solving, understanding language, and recognizing patterns.\n\n- **Applications**: AI is used in various fields, including healthcare (diagnosis and treatmen

In [36]:
memory.load_memory_variables({})['history']

[HumanMessage(content='What are the first four colors of a rainbow?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The first four colors of a rainbow are red, orange, yellow, and green.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='and the other 3?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The other three colors of a rainbow are blue, indigo, and violet.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Explain AI in 2 bullet points', additional_kwargs={}, response_metadata={}),
 AIMessage(content='- **Definition**: Artificial Intelligence (AI) refers to the simulation of human intelligence in machines programmed to think and learn, enabling them to perform tasks that typically require human cognitive functions, such as problem-solving, understanding language, and recognizing patterns.\n\n- **Applications**: AI is used in various fields, including healthcare (diagnosis and treatment recommendations

In [38]:
query = {'query': 'Now do the same for Deep Learning'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

- **Definition**: Deep Learning is a subset of machine learning that uses neural networks with many layers (deep neural networks) to model complex patterns in large datasets, enabling machines to learn from data in a way that mimics human brain function.

- **Applications**: Deep Learning is widely used in image and speech recognition, natural language processing, autonomous vehicles, and various other fields requiring advanced data analysis and pattern recognition.


In [39]:
memory.load_memory_variables({})['history']

[HumanMessage(content='What are the first four colors of a rainbow?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The first four colors of a rainbow are red, orange, yellow, and green.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='and the other 3?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The other three colors of a rainbow are blue, indigo, and violet.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Explain AI in 2 bullet points', additional_kwargs={}, response_metadata={}),
 AIMessage(content='- **Definition**: Artificial Intelligence (AI) refers to the simulation of human intelligence in machines programmed to think and learn, enabling them to perform tasks that typically require human cognitive functions, such as problem-solving, understanding language, and recognizing patterns.\n\n- **Applications**: AI is used in various fields, including healthcare (diagnosis and treatment recommendations

In [40]:
query = {'query': 'What have we discussed so far?'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

So far, we have discussed the following topics:

1. The first four colors of a rainbow (red, orange, yellow, green) and the remaining three (blue, indigo, violet).
2. A brief explanation of Artificial Intelligence (AI) in two bullet points.
3. A brief explanation of Deep Learning in two bullet points.


In [41]:
memory.load_memory_variables({})['history']

[HumanMessage(content='What are the first four colors of a rainbow?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The first four colors of a rainbow are red, orange, yellow, and green.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='and the other 3?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The other three colors of a rainbow are blue, indigo, and violet.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Explain AI in 2 bullet points', additional_kwargs={}, response_metadata={}),
 AIMessage(content='- **Definition**: Artificial Intelligence (AI) refers to the simulation of human intelligence in machines programmed to think and learn, enabling them to perform tasks that typically require human cognitive functions, such as problem-solving, understanding language, and recognizing patterns.\n\n- **Applications**: AI is used in various fields, including healthcare (diagnosis and treatment recommendations

## Conversation Chains with ConversationBufferWindowMemory

If you have a really long conversation, you might exceed the max token limit of the context window allowed for the LLM when using `ConversationBufferMemory` so `ConversationBufferWindowMemory` helps in just storing the last K conversations (one conversation piece is one user message and the corresponding AI message from the LLM) and thus helps you manage token limits and costs

In [42]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferWindowMemory
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

SYS_PROMPT = """Act as a helpful assistant and give brief answers"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", SYS_PROMPT),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{query}"),
    ]
)

# stores last 2 sets of human-AI conversations
memory = ConversationBufferWindowMemory(return_messages=True, k=2)

# creating our conversation chain now
def get_memory_messages(query):
    return memory.load_memory_variables(query)['history']

conversation_chain = (
    RunnablePassthrough.assign(
        history=RunnableLambda(get_memory_messages)
    ) # sends current query (input by user at runtime) and history messages to next step
      |
    prompt # creates prompt using the previous two variables
      |
    chatgpt # generates response using the prompt from previous step
)

/var/folders/8v/xkrl1q210t5_4t4hvbx286800000gp/T/ipykernel_5092/96416143.py:15: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(return_messages=True, k=2)


In [43]:
query = {'query': 'What are the first four colors of a rainbow?'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

The first four colors of a rainbow are red, orange, yellow, and green.


In [44]:
query = {'query': 'and the other 3?'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

The other three colors of a rainbow are blue, indigo, and violet.


In [45]:
query = {'query': 'Explain AI in 2 bullet points'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

- **Definition**: Artificial Intelligence (AI) refers to the simulation of human intelligence in machines programmed to think and learn, enabling them to perform tasks that typically require human cognitive functions, such as problem-solving, understanding language, and recognizing patterns.

- **Applications**: AI is used in various fields, including healthcare (diagnosis and treatment recommendations), finance (fraud detection and algorithmic trading), and everyday technology (virtual assistants and recommendation systems).


In [46]:
query = {'query': 'Now do the same for Deep Learning'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

- **Definition**: Deep Learning is a subset of machine learning that uses neural networks with many layers (deep neural networks) to model complex patterns in large datasets, enabling machines to learn from data in a way that mimics human brain function.

- **Applications**: Deep Learning is widely used in image and speech recognition, natural language processing, autonomous vehicles, and various other fields where large amounts of unstructured data are processed.


In [47]:
memory.load_memory_variables({})['history']

[HumanMessage(content='Explain AI in 2 bullet points', additional_kwargs={}, response_metadata={}),
 AIMessage(content='- **Definition**: Artificial Intelligence (AI) refers to the simulation of human intelligence in machines programmed to think and learn, enabling them to perform tasks that typically require human cognitive functions, such as problem-solving, understanding language, and recognizing patterns.\n\n- **Applications**: AI is used in various fields, including healthcare (diagnosis and treatment recommendations), finance (fraud detection and algorithmic trading), and everyday technology (virtual assistants and recommendation systems).', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Now do the same for Deep Learning', additional_kwargs={}, response_metadata={}),
 AIMessage(content='- **Definition**: Deep Learning is a subset of machine learning that uses neural networks with many layers (deep neural networks) to model complex patterns in large datasets, 

In [48]:
query = {'query': 'What have we discussed so far?'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

So far, we have discussed the following topics:

1. A brief explanation of Artificial Intelligence (AI) in two bullet points, covering its definition and applications.
2. A brief explanation of Deep Learning in two bullet points, including its definition and applications.


In [49]:
memory.load_memory_variables({})['history']

[HumanMessage(content='Now do the same for Deep Learning', additional_kwargs={}, response_metadata={}),
 AIMessage(content='- **Definition**: Deep Learning is a subset of machine learning that uses neural networks with many layers (deep neural networks) to model complex patterns in large datasets, enabling machines to learn from data in a way that mimics human brain function.\n\n- **Applications**: Deep Learning is widely used in image and speech recognition, natural language processing, autonomous vehicles, and various other fields where large amounts of unstructured data are processed.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What have we discussed so far?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='So far, we have discussed the following topics:\n\n1. A brief explanation of Artificial Intelligence (AI) in two bullet points, covering its definition and applications.\n2. A brief explanation of Deep Learning in two bullet points, incl

### Conversation Chains with ConversationSummaryMemory

If you have a really long conversation or a lot of messages, you might exceed the max token limit of the context window allowed for the LLM when using `ConversationBufferMemory`

`ConversationSummaryMemory` creates a summary of the conversation history over time. This can be useful for condensing information from the conversation messages over time.

This memory is most useful for longer conversations, where keeping the past message history in the prompt verbatim would take up too many tokens.

In [61]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationSummaryMemory
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

SYS_PROMPT = """Act as a helpful assistant and give brief answers"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", SYS_PROMPT),
        # MessagesPlaceholder(variable_name="history") - This placeholder inserts conversation history messages into the prompt
        # The variable_name can be any string - "history", "history_summary", "chat_history", etc.
        # It will be populated by the memory.load_memory_variables() function with the corresponding key
        MessagesPlaceholder(variable_name="history_summary"),
        ("human", "{query}"),
    ]
)

memory = ConversationSummaryMemory(return_messages=True, llm=chatgpt)
# creating our conversation chain now
def get_memory_messages(query):
    return memory.load_memory_variables(query)['history']

conversation_chain = (
    RunnablePassthrough.assign(
        history_summary=RunnableLambda(get_memory_messages)
    ) # sends current query (input by user at runtime) and history messages as a summary to next step
      |
    prompt # creates prompt using the previous two variables
      |
    chatgpt # generates response using the prompt from previous step
)

In [62]:
query = {'query': 'Explain AI in 2 bullet points'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

- **Definition**: Artificial Intelligence (AI) refers to the simulation of human intelligence processes by machines, particularly computer systems, enabling them to perform tasks such as learning, reasoning, and problem-solving.

- **Applications**: AI is used in various fields, including healthcare (diagnosis and treatment recommendations), finance (fraud detection), autonomous vehicles, customer service (chatbots), and more, enhancing efficiency and decision-making.


In [63]:
query = {'query': 'Now do the same for Deep Learning'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

- Deep Learning is a subset of machine learning that uses neural networks with many layers to analyze and learn from large amounts of data, enabling complex pattern recognition and decision-making.
- It is widely used in applications such as image and speech recognition, natural language processing, and autonomous systems, significantly improving performance in these areas.


In [64]:
memory.load_memory_variables({})

{'history': [SystemMessage(content='The human asks the AI to explain artificial intelligence in two bullet points. The AI defines artificial intelligence as the simulation of human intelligence processes by machines, enabling tasks like learning and problem-solving, and lists its applications in fields such as healthcare, finance, autonomous vehicles, and customer service, highlighting its role in enhancing efficiency and decision-making. The human then requests a similar explanation for Deep Learning, and the AI describes it as a subset of machine learning that uses neural networks with many layers to analyze large amounts of data for complex pattern recognition and decision-making, with applications in image and speech recognition, natural language processing, and autonomous systems, significantly improving performance in these areas.', additional_kwargs={}, response_metadata={})]}

## Conversation Chains with VectorStoreRetrieverMemory

`VectorStoreRetrieverMemory` stores historical conversation messages in a vector store and queries the top-K most "relevant" history messages every time it is called.

This differs from most of the other Memory classes in that it doesn't explicitly track the order of interactions but retrieves history based on embedding similarity to the current question or prompt.

In this case, the "docs" are previous conversation snippets. This can be useful to refer to relevant pieces of information that the AI was told earlier in the conversation.

### Connect to  Open AI Embedding Models

LangChain enables us to access Open AI embedding models which include the newest models: a smaller and highly efficient `text-embedding-3-small` model, and a larger and more powerful `text-embedding-3-large` model.

In [65]:
from langchain_openai import OpenAIEmbeddings

# details here: https://openai.com/blog/new-embedding-models-and-api-updates
openai_embed_model = OpenAIEmbeddings(model='text-embedding-3-small')

#### Create a Vector Database to store conversation history

Here we use the Chroma vector DB and initialize an empty database collection to store conversation messages

In [66]:
from langchain_chroma import Chroma

# create empty vector DB
chroma_db = Chroma(collection_name='history_db',
                   embedding_function=openai_embed_model)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [67]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import VectorStoreRetrieverMemory
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

SYS_PROMPT = """Act as a helpful assistant and give brief answers"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", SYS_PROMPT),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{query}"),
    ]
)

# load 2 most similar conversation messages from vector db history for each new message \ prompt
# this uses cosine embedding similarity to load the top 2 similar messgages to the input prompt \ query
retriever = chroma_db.as_retriever(search_type="similarity",
                                   search_kwargs={"k": 2})
memory = VectorStoreRetrieverMemory(retriever=retriever, return_messages=True)

# creating our conversation chain now
def get_memory_messages(query):
    return [memory.load_memory_variables(query)['history']]

conversation_chain = (
    RunnablePassthrough.assign(
        history=RunnableLambda(get_memory_messages)
    ) # sends current query (input by user at runtime) and history messages to next step
      |
    prompt # creates prompt using the previous two variables
      |
    chatgpt # generates response using the prompt from previous step
)

/var/folders/8v/xkrl1q210t5_4t4hvbx286800000gp/T/ipykernel_5092/573889585.py:18: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = VectorStoreRetrieverMemory(retriever=retriever, return_messages=True)


In [68]:
query = {'query': 'Tell me about AI'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


Artificial Intelligence (AI) refers to the simulation of human intelligence in machines programmed to think and learn. It encompasses various technologies, including machine learning, natural language processing, and robotics. AI can perform tasks such as speech recognition, decision-making, and problem-solving, and is used in applications like virtual assistants, autonomous vehicles, and data analysis.


In [69]:
query = {'query': 'What about deep learning'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

Number of requested results 2 is greater than number of elements in index 1, updating n_results = 1


Deep learning is a subset of machine learning that uses neural networks with many layers (deep neural networks) to analyze and learn from large amounts of data. It excels in tasks such as image and speech recognition, natural language processing, and game playing. Deep learning models automatically extract features from raw data, making them highly effective for complex problems.


In [70]:
query = {'query': 'Tell me about the fastest animal in the world in 2 lines'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

The fastest animal in the world is the peregrine falcon, which can reach speeds over 240 mph (386 km/h) during its hunting stoop (high-speed dive). On land, the cheetah holds the record, capable of sprinting up to 60-70 mph (97-113 km/h) in short bursts.


In [71]:
query = {'query': 'What about the cheetah?'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

The cheetah is the fastest land animal, capable of reaching speeds of 60-70 mph (97-113 km/h) in short bursts covering distances up to 1,500 feet. Its unique adaptations, such as a lightweight body, long legs, and a flexible spine, enable it to accelerate rapidly and make sharp turns while chasing prey.



Now for a new query around machine learning even if the most recent conversation messages have been about animals, it uses the vector databases to load the last 2 historical conversations which are closest to the current question in terms of semantic similarity

In [72]:
print(memory.load_memory_variables({'query': 'What about machine learning?'})['history'])

query: What about deep learning
output: Deep learning is a subset of machine learning that uses neural networks with many layers (deep neural networks) to analyze and learn from large amounts of data. It excels in tasks such as image and speech recognition, natural language processing, and game playing. Deep learning models automatically extract features from raw data, making them highly effective for complex problems.
query: Tell me about AI
output: Artificial Intelligence (AI) refers to the simulation of human intelligence in machines programmed to think and learn. It encompasses various technologies, including machine learning, natural language processing, and robotics. AI can perform tasks such as speech recognition, decision-making, and problem-solving, and is used in applications like virtual assistants, autonomous vehicles, and data analysis.


In [73]:
query = {'query': 'What about machine learning?'}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content}) # remember to save your current conversation in memory
print(response.content)

Machine learning is a subset of artificial intelligence that focuses on the development of algorithms that allow computers to learn from and make predictions or decisions based on data. It involves training models on datasets to identify patterns and improve performance over time without being explicitly programmed. Machine learning is used in various applications, including recommendation systems, fraud detection, and predictive analytics.


In [75]:
print(memory.load_memory_variables({'query': 'What is the capital of India?'})['history'])

query: What about the cheetah?
output: The cheetah is the fastest land animal, capable of reaching speeds of 60-70 mph (97-113 km/h) in short bursts covering distances up to 1,500 feet. Its unique adaptations, such as a lightweight body, long legs, and a flexible spine, enable it to accelerate rapidly and make sharp turns while chasing prey.
query: Tell me about AI
output: Artificial Intelligence (AI) refers to the simulation of human intelligence in machines programmed to think and learn. It encompasses various technologies, including machine learning, natural language processing, and robotics. AI can perform tasks such as speech recognition, decision-making, and problem-solving, and is used in applications like virtual assistants, autonomous vehicles, and data analysis.


## Multi-user Conversation Chains with ChatMessageHistory

The concept of `ChatHistory` refers to a class in LangChain which can be used to wrap an arbitrary chain. This `ChatHistory` will keep track of inputs and outputs of the underlying chain, and append them as messages to a message database. Future interactions will then load those messages and pass them into the chain as part of the input.

The beauty of `ChatMessageHistory` is that we can store separate conversation histories per user or session which is often the need for real-world chatbots which will be accessed by many users at the same time.

We use a `get_session_history` function which is expected to take in a `session_id` and return a Message History object. Everything is stored in memory. This `session_id` is used to distinguish between separate conversations, and should be passed in as part of the config when calling the new chain


- `ChatMessageHistory`: This is a class in LangChain that stores a sequence of chat messages (such as HumanMessage, AIMessage, SystemMessage) for a conversation. It allows you to append new messages and retrieve the full message history, which is useful for maintaining conversational context.

- `BaseChatMessageHistory`: This is an abstract base class that defines the interface for chat message history storage backends. It specifies methods like add_message, get_messages, and clear, which concrete implementations (like ChatMessageHistory, RedisChatMessageHistory, etc.) must provide.

- `RunnableWithMessageHistory`: This is a wrapper for a Runnable (such as a chain or LLM) that automatically manages message history for each session or user. It uses a function (like get_session_history) to retrieve the appropriate message history object based on a session ID, and injects the history into the chain's input. This enables multi-user or multi-session conversational experiences.

- `MessagesPlaceholder`: This is a special prompt template variable in LangChain that acts as a placeholder for a list of messages (e.g., the conversation history). When rendering a prompt, MessagesPlaceholder is replaced with the actual messages from history, allowing the LLM to see the full conversation context.


In [76]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# used to retrieve conversation history from memory
# based on a specific user or session ID
history_store = {}
def get_session_history(session_id):
    if session_id not in history_store:
        history_store[session_id] = ChatMessageHistory()
    return history_store[session_id]

# prompt to load in history and current input from the user
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Act as a helpful AI Assistant"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{human_input}"),
    ]
)

# create a basic LLM Chain
llm_chain = (prompt_template
                |
             chatgpt)

# create a conversation chain which can load memory based on specific user or session id
conv_chain = RunnableWithMessageHistory(
    llm_chain,
    get_session_history,
    input_messages_key="human_input",
    history_messages_key="history",
)

# create a utility function to take in current user input prompt and their session ID
# streams result live back to the user from the LLM
def chat_with_llm(prompt: str, session_id: str):
    for chunk in conv_chain.stream({"human_input": prompt},
                                   {'configurable': { 'session_id': session_id}}):
        print(chunk.content, end="")

Test conversation chain for user 1

In [77]:
user_id = 'bob123'
prompt = "Hi I am Bob, can you explain AI in 3 bullet points?"
chat_with_llm(prompt, user_id)

Sure, Bob! Here are three key points about AI:

1. **Definition and Purpose**: Artificial Intelligence (AI) refers to the simulation of human intelligence in machines that are programmed to think and learn. Its primary purpose is to perform tasks that typically require human intelligence, such as problem-solving, understanding natural language, and recognizing patterns.

2. **Types of AI**: AI can be categorized into two main types: Narrow AI, which is designed for specific tasks (like virtual assistants or recommendation systems), and General AI, which aims to perform any intellectual task that a human can do. Currently, most AI applications are examples of Narrow AI.

3. **Applications and Impact**: AI is used across various industries, including healthcare (for diagnostics), finance (for fraud detection), and transportation (in self-driving cars). Its impact is significant, as it enhances efficiency, drives innovation, and can lead to new solutions for complex problems, but it also 

In [78]:
prompt = "Now do the same for deep learning"
chat_with_llm(prompt, user_id)

Of course! Here are three key points about deep learning:

1. **Definition and Structure**: Deep learning is a subset of machine learning that uses neural networks with many layers (hence "deep") to model complex patterns in large datasets. These neural networks are inspired by the structure and function of the human brain, allowing them to learn from vast amounts of data.

2. **Training Process**: Deep learning models are trained using large datasets and require significant computational power. They learn by adjusting the weights of connections in the network through a process called backpropagation, which minimizes the difference between the predicted output and the actual output.

3. **Applications and Advancements**: Deep learning has led to breakthroughs in various fields, including image and speech recognition, natural language processing, and autonomous systems. Its ability to automatically extract features from raw data has made it a powerful tool for tasks like facial recognit

In [79]:
prompt = "Discuss briefly what have we discussed so far is bullet points?"
chat_with_llm(prompt, user_id)

Sure! Here’s a brief summary of what we’ve discussed so far in bullet points:

- **Artificial Intelligence (AI)**:
  - AI simulates human intelligence in machines to perform tasks like problem-solving and language understanding.
  - It is categorized into Narrow AI (specific tasks) and General AI (human-like intelligence).
  - AI is widely used in various industries, enhancing efficiency and innovation while raising ethical concerns.

- **Deep Learning**:
  - Deep learning is a subset of machine learning that uses multi-layered neural networks to model complex patterns.
  - It involves training on large datasets using backpropagation to adjust network weights.
  - Deep learning has enabled advancements in fields like image recognition, speech processing, and natural language understanding.

Now test conversation chain for user 2

In [80]:
user_id = 'james007'
prompt = "Hi can you explain what is an LLM in 2 bullet points?"
chat_with_llm(prompt, user_id)

Sure! Here are two key points about LLMs (Large Language Models):

- **Definition**: LLMs are advanced artificial intelligence models designed to understand and generate human-like text based on vast amounts of data. They utilize deep learning techniques, particularly transformer architectures, to process and predict language patterns.

- **Applications**: LLMs are used in various applications, including chatbots, content generation, language translation, and summarization, enabling more natural interactions between humans and machines.

In [81]:
prompt = "Actually I meant in the context of AI?"
chat_with_llm(prompt, user_id)

Got it! Here’s a refined explanation of LLMs in the context of AI:

- **Definition**: In AI, LLMs (Large Language Models) are sophisticated neural network models trained on extensive datasets to understand, generate, and manipulate human language. They leverage deep learning techniques, particularly transformer architectures, to capture complex language patterns and semantics.

- **Capabilities**: LLMs can perform a wide range of language-related tasks, such as answering questions, writing essays, translating languages, and engaging in conversations, making them valuable tools in natural language processing (NLP) applications.

In [82]:
prompt = "Summarize briefly what we have discussed so far?"
chat_with_llm(prompt, user_id)

We discussed Large Language Models (LLMs) in the context of AI, highlighting two main points: 

1. **Definition**: LLMs are advanced neural network models trained on large datasets to understand and generate human language, utilizing deep learning techniques like transformers.
  
2. **Capabilities**: They can perform various language-related tasks, such as answering questions, writing content, translating languages, and engaging in conversations, making them essential in natural language processing applications.

In [83]:
user_id = 'bob123'
prompt = "Discuss briefly what have we discussed so far is bullet points?"
chat_with_llm(prompt, user_id)

Certainly! Here’s a concise summary of our discussion in bullet points:

- **Artificial Intelligence (AI)**:
  - Simulates human intelligence in machines for tasks like problem-solving and language understanding.
  - Divided into Narrow AI (specific tasks) and General AI (human-like intelligence).
  - Used across industries, enhancing efficiency and innovation, while raising ethical concerns.

- **Deep Learning**:
  - A subset of machine learning using multi-layered neural networks to model complex patterns.
  - Trained on large datasets through backpropagation to optimize performance.
  - Drives advancements in image recognition, speech processing, and natural language understanding.

In [84]:
user_id = 'Sourav'
prompt = "Discuss briefly what have we discussed so far is bullet points?"
chat_with_llm(prompt, user_id)

Sure! However, I don't have access to previous conversations or discussions. If you could provide a brief summary or key points from our previous discussions, I can help you organize them into bullet points.

## Multi-user Window-based Conversation Chains with persistence - SQLChatMessageHistory

The beauty of `SQLChatMessageHistory` is that we can store separate conversation histories per user or session which is often the need for real-world chatbots which will be accessed by many users at the same time. Instead of in-memory we can store it in a SQL database which can be used to store a lot of conversations.

We use a `get_session_history` function which is expected to take in a `session_id` and return a Message History object. Everything is stored in a SQL database. This `session_id` is used to distinguish between separate conversations, and should be passed in as part of the config when calling the new chain

We also use a `memory_buffer_window` function to only use the top-K last historical conversations before sending it to the LLM, basically our own implementation of `ConversationBufferWindowMemory`

In [85]:
# removes the memory database file - usually not needed
# you can run this only when you want to remove all conversation histories
!rm memory.db

rm: memory.db: No such file or directory


In [86]:
from langchain_community.chat_message_histories import SQLChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough

# used to retrieve conversation history from database
# based on a specific user or session ID
def get_session_history_db(session_id):
    return SQLChatMessageHistory(session_id, "sqlite:///memory.db")

# prompt to load in history and current input from the user
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Act as a helpful AI Assistant"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{human_input}"),
    ]
)

# create a memory buffer window function to return the last K conversations
def memory_buffer_window(messages, k=2):
    return messages[-(k+1):]

# create a basic LLM Chain which only sends the last K conversations per user
llm_chain = (
    RunnablePassthrough.assign(history=lambda x: memory_buffer_window(x["history"]))
      |
    prompt_template
      |
    chatgpt
)

In [87]:
# create a conversation chain which can load memory based on specific user or session id
conv_chain = RunnableWithMessageHistory(
    llm_chain,
    get_session_history_db,
    input_messages_key="human_input",
    history_messages_key="history",
)

# create a utility function to take in current user input prompt and their session ID
# streams result live back to the user from the LLM
def chat_with_llm(prompt: str, session_id: str):
    for chunk in conv_chain.stream({"human_input": prompt},
                                   {'configurable': { 'session_id': session_id}}):
        print(chunk.content, end="")

Test conversation chain for user 1

In [88]:
user_id = 'jim001'
prompt = "Hi can you tell me which is the fastest animal?"
chat_with_llm(prompt, user_id)

/Users/sourav.banerjee/Documents/My Codebases/GenerativAI_Demystified/.venv/lib/python3.11/site-packages/langchain_core/runnables/history.py:596: LangChainDeprecationWarning: `connection_string` was deprecated in LangChain 0.2.2 and will be removed in 1.0. Use connection instead.
  message_history = self.get_session_history(


The fastest animal in the world is the peregrine falcon. When in a dive, it can reach speeds of over 240 miles per hour (386 kilometers per hour). If you're considering speed in level flight, the Brazilian free-tailed bat holds the record, flying at speeds of around 99 miles per hour (160 kilometers per hour). For land animals, the cheetah is the fastest, capable of running speeds up to 60-70 miles per hour (97-113 kilometers per hour) in short bursts.

In [89]:
prompt = "what about the slowest animal?"
chat_with_llm(prompt, user_id)

The slowest animal is often considered to be the three-toed sloth. It moves at an average speed of about 0.24 kilometers per hour (0.15 miles per hour) when climbing trees. In water, they can swim slightly faster, but they are still quite slow compared to many other animals. Another contender for the title of slowest animal is the garden snail, which moves at a speed of about 0.03 miles per hour (0.048 kilometers per hour). Both of these animals have adapted to their slow pace as part of their survival strategies.

In [90]:
prompt = "what about the largest animal?"
chat_with_llm(prompt, user_id)

The largest animal on Earth is the blue whale (*Balaenoptera musculus*). Blue whales can reach lengths of up to 100 feet (30 meters) or more and can weigh as much as 200 tons (approximately 181 metric tonnes). They are not only the largest animals alive today but also the largest animals known to have ever existed on our planet. Their immense size is supported by their aquatic environment, which allows them to float and move more easily than terrestrial animals of similar size.

In [91]:
prompt = "what topics have we discussed, show briefly as bullet points"
chat_with_llm(prompt, user_id)

Sure! Here are the topics we've discussed so far:

- The slowest animal (three-toed sloth and garden snail)
- The largest animal (blue whale)

Now test conversation chain for user 2

In [92]:
user_id = 'john005'
prompt = "Explain AI in 3 bullets to a child"
chat_with_llm(prompt, user_id)

Sure! Here are three simple points to explain AI to a child:

1. **Smart Helpers**: AI is like a smart robot or computer that can help us with tasks, like answering questions or playing games, just like a friend would!

2. **Learning from Experience**: AI learns from lots of information, kind of like how you learn from your school lessons and practice. The more it learns, the better it gets at helping us.

3. **Talking and Understanding**: AI can understand what we say or write and can even talk back to us, making it feel like we’re having a conversation with a really clever buddy!

In [93]:
prompt = "Now do the same for Generative AI"
chat_with_llm(prompt, user_id)

Of course! Here are three simple points to explain Generative AI to a child:

1. **Creative Robot**: Generative AI is like a creative robot that can make new things, like pictures, stories, or music, all by itself, just like an artist or a writer!

2. **Using Ideas**: It learns from lots of examples, like reading many books or looking at many paintings, so it can come up with its own cool ideas based on what it has learned.

3. **Imagination in Action**: When you ask Generative AI to create something, it uses its imagination to come up with fun and interesting stuff, almost like magic!

In [94]:
prompt = "Now do the same for machine learning"
chat_with_llm(prompt, user_id)

Sure! Here are three simple points to explain machine learning to a child:

1. **Learning Like You**: Machine learning is when computers learn from examples, just like you learn new things in school by practicing and seeing how things work.

2. **Getting Better Over Time**: The more examples the computer sees, the better it gets at understanding and making decisions, like how you get better at a game the more you play it.

3. **Solving Problems**: With machine learning, computers can help solve problems or make predictions, like guessing what movie you might like based on the ones you’ve watched before!

In [95]:
prompt = "what topics have we discussed, show briefly as bullet points"
chat_with_llm(prompt, user_id)

Sure! Here are the topics we've discussed so far:

- **Generative AI**
  - Creative robot that makes new things (pictures, stories, music)
  - Learns from examples to come up with new ideas
  - Uses imagination to create fun and interesting content

- **Machine Learning**
  - Computers learn from examples like humans do
  - Improves over time with more practice
  - Helps solve problems and make predictions based on data

If you have more questions or need further information on these topics, feel free to ask!